In [2]:
import numpy as np 
from scipy.stats import norm
from scipy.linalg import cholesky

$$
\mathbb{P}\left(Y \in \cdot \mid Y \in \mathrm{S}_{\mu^{(t)}, \mathbf{i}}\right) \text { et } \mathbb{P}\left(Y \in \cdot \mid\left[\mu^{(t)}\right]^T Y=s\right)
$$
Pour le premier cas : on définit la strat, on tire un point au hasard dans la strat, et ensuite on échantillonne conditionnellement à ce point !
On le réécrit : 
$$
X=v^{\top} \xi \sim N\left(0, v^{\top} \Sigma v\right)=N(0,1)
$$
On a la distrbution conditionnelle : 

$$
(\xi \mid X=x) \sim N\left(\frac{\Sigma v}{v^{\top} \Sigma v} x, \Sigma-\frac{\Sigma v v^{\top} \Sigma}{v^{\top} \Sigma v}\right)=N\left(\Sigma v x, \Sigma-\Sigma v v^{\top} \Sigma\right)
$$
On utilise le tips ci-dessous pour pouvoir simuler (décomposition du terme en variance)

$$
\begin{aligned}
& \left(A-\Sigma v v^{\top} A\right)\left(A-\Sigma v v^{\top} A\right)^{\top} \\
& \quad=A A^{\top}-A A^{\top} v v^{\top} \Sigma-\Sigma v v^{\top} A A^{\top}+\Sigma v v^{\top} \Sigma v v^{\top} \Sigma \\
& \quad=\Sigma-\Sigma v v^{\top} \Sigma
\end{aligned}
$$

Suivant que l'on veuille sample avec ou non une condition sur la valeur de X, on le spécifie dans la fonction ci-dessous.

$$
\begin{aligned}
& \text { for } i=1, \ldots, K \\
& \quad \text { generate } U \sim \operatorname{Unif}[0,1] \\
& \quad V \leftarrow(i-1+U) / K \\
& \quad X \leftarrow \Phi^{-1}(V) \\
& \quad \text { generate } Z \sim N(0, I) \text { in } \Re^d \\
& \quad \xi \leftarrow \Sigma v X+\left(A-\Sigma v v^{\top} A\right) Z
\end{aligned}
$$

In [3]:
def stratified_sampling_linear_projection(mu, Sigma, v, K, x=None):
    """
    Generates K samples from N(0, Sigma) stratified along the direction determined by v.
    
    Args:
    - mu (array): The mean vector of the distribution.
    - Sigma (2D array): The covariance matrix of the distribution.
    - v (array): The vector along which stratification is done.
    - K (int): The number of stratified samples to generate.
    - x (optionnal) :  si on donne une valeur pour x alors c'est qu'on conditionne à une valeur particulière de x. Sinon on sample sur tout la strat Si. Taille K
    
    Returns:
    - samples (2D array): The generated stratified samples. (K, de N(0,sigma), stratifié sur la direction donnée par v)
    """
    d = len(mu)  # Dimension of the normal distribution
    # Normalize v so that v^T Sigma v = 1
    v = v / np.sqrt(v.T @ Sigma @ v)
    
    # Generate K stratified samples for the standard normal distribution along v
    U = np.random.uniform(0, 1, K)     # Uniformly distributed samples in (0,1)
    V = (np.arange(K) + U) / K         # Stratified samples in (0,1)
    
    X = norm.ppf(V)                    # Inverse CDF (quantile function) to get stratified samples for N(0, 1)
    
    # Compute the matrix A for the conditional distribution of xi given x
    A = cholesky(Sigma, lower=True)    # Cholesky factorization
    A_minus_v_Sigma_v_T_A = A - np.outer(Sigma @ v, v.T @ A)
    
    # Generate K samples from the conditional distribution of xi given X
    Z = np.random.randn(K, d)  # Z ~ N(0, I) in d dimensions 
    if x is not None:

        xi_samples = Sigma @ v * x[:, None] + (A_minus_v_Sigma_v_T_A @ Z.T).T  # Conditional samples 
        #xi_samples = Sigma @ v @ x + (A_minus_v_Sigma_v_T_A @ Z.T).T

    else:
        xi_samples = Sigma @ v * X[:, None] + (A_minus_v_Sigma_v_T_A @ Z.T).T 
        #xi_samples = Sigma @ v @ X + (A_minus_v_Sigma_v_T_A @ Z.T).T 
        
    
    return xi_samples + mu             # Add the mean to each sample

In [4]:
def tirer_K_nb(borne_inf,borne_sup,K):
    #vérifier comment tirer les nonbres uniformes sur -inf, a et sur a, +inf. Plusieurs manieres de faire je suis pas sur 
    if borne_inf == -np.inf:
        # Pour l'intervalle (-∞, b)

        u = np.random.uniform(0, 1, K)
        x_neg_inf = borne_sup - np.log(1 / u)
        return x_neg_inf

    elif borne_sup == np.inf:
        u = np.random.uniform(0, 1, K)
        x_pos_inf = borne_inf + np.log(1 / (1 - u))
        return x_pos_inf

    else:
        out = np.random.uniform(borne_inf , borne_sup, K)
        return out 

Tirage si dessous suivant une valeur de $\mathbb{P}\left(Y \in \cdot \mid\left[\mu_k^{(t)}\right]^T Y=s\right)$

Ici $s \in\left\{G_k^{-1}(1 / I), \cdots, G_k^{-1}((I-1) / I)\right\}$

In [14]:

def quantiles_normaux(I):
    """Args : I 
    return : les quantiles normaux, affiches ci-dessus;"""
   
    quantiles = [norm.ppf(k / I) for k in range(1, I)]
    
    return quantiles


In [8]:
# Example usage:
mu = np.array([1, 2, 3])               # Mean vector - on est en 3 dimensions 
Sigma = np.array([[1, 0.5, 0.1],       # Covariance matrix
                  [0.5, 2, 0.3],
                  [0.1, 0.3, 1]])
v = np.array([1, 1, 1])                # Vector for stratification which is centerd 
K = 2                           # Number of samples to generate - 10 strates. Donc pour chaque strat les 3 coordonnées. 

# Generate stratified samples
samples = stratified_sampling_linear_projection(mu, Sigma, v, K,np.array([0.27,0.28]))
#samples  # Display first 5 samples for brevity
samples

array([[1.88841998, 1.26680687, 3.49501925],
       [0.73769309, 2.79757164, 3.13906456]])

Calcul de 
$$
\mathbb{P}\left(Y \in \cdot \mid Y \in \mathrm{S}_{\mu^{(t)}, \mathbf{i}}\right)
$$

on donne juste le i, numéro de la strat sur laquelle on veut tirer 

In [9]:
def tirer_y_strat_nu(v,i,mu,sigma,K,S):
    """

    Args : 
        - v : l'orientation de la strat 
        - i : indice appartenant à 0....I-1 !! et pas 1,....I ATTENTION 
        - mu : moyenne de la v.a. Y 
        - sigma : variance de la v.a. Y 
        - S : nos strats 
    
    Outputs : tire K variables suivant l'indice i pour la strat donnée """

    return  stratified_sampling_linear_projection(mu, Sigma, v, K,tirer_K_nb(S[i][0],S[i][1],K))

    

# Calcul de Si, la strat Si

$$
\mathrm{S}_{\mathbf{i}} \stackrel{\text { def }}{=} \prod_{k=1}^m\left(G_k^{-1}\left(\frac{i_k-1}{I}\right), G_k^{-1}\left(\frac{i_k}{I}\right)\right]
$$
On calcul Si, et on va tirer dedans. 

In [10]:

def strats_Si(I):
    # Calcul de Si pour chaque ik de 1 à I
    results = [(norm.ppf((ik - 1) / I), norm.ppf(ik / I)) for ik in range(1, I + 1)]
    return results

# Afficher les premiers et derniers éléments pour vérification
results = strats_Si(100) #les strats que l'on utilise ds la suite 
results[:5], results[-5:]

#-inf*mu -3$mu


def p_i_mu(I):
    return 1/I

# Tirer un point suivant une strat 

$$
\mathbb{P}\left(Y \in \cdot \mid Y \in \mathrm{S}_{\mu^{(t)}, \mathbf{i}}\right)
$$

In [11]:
tirer_y_strat_nu(v,99,mu,Sigma,K,results) #le nombre en dur est le numero de la strat ici de 0 à 99 car j'ai fais avec 100 levels dans l'exemple 

array([[2.94047713, 3.64473484, 6.10313107],
       [3.08717261, 7.66747004, 3.80231375]])

# Choix du vecteur d'allocation 

$$
\sigma_{\mathbf{i}}^2(\mu) \stackrel{\text { def }}{=} \mathrm{E}\left[\phi^2(Y) \mid \mu^T Y \in \mathrm{S}_{\mathbf{i}}\right]-\left(\mathrm{E}\left[\phi(Y) \mid \mu^T Y \in \mathrm{S}_{\mathbf{i}}\right]\right)^2
$$

Puis cela permet de calculer : 

$$
q_{\mathbf{i}}^{\star}(\mu) \stackrel{\text { def }}{=} \frac{p_{\mathbf{i}}(\mu) \sigma_{\mathbf{i}}(\mu)}{\sum_{\mathbf{j} \in \mathcal{I}} p_{\mathbf{j}}(\mu) \sigma_{\mathbf{j}}(\mu)} .
$$

In [31]:
#fonctions payoff arbitraires 

def phi(x):
    return x 

calcul du sigma i tel que défini ci-dessus. 

In [89]:
def sigma_i(v,i,mu,Sigma,K,results):
    #calcul simplement le sigma_i que l'on utilise. Sachant que le pi vaut 1/I 
    return np.sqrt(np.mean(phi(tirer_y_strat_nu(v,i,mu,Sigma,K,results))**2)- np.mean(phi(tirer_y_strat_nu(v,i,mu,Sigma,K,results)))**2)

In [90]:
sigma_i(v,0,mu,Sigma,2000,results)

1.7096837812190677

calcul du vecteur d'allocation optimale à l'instant initial. 

In [83]:
def calculate_q_star(mu, I, v, Sigma, K, results):
    # Initialisation du vecteur des sigmas
    sigma_values = np.zeros(I)
    
    # Calcul de chaque sigma_i pour un mu donné
    for i in range(I):
        sigma_values[i] = sigma_i(v, i, mu, Sigma, K, results)
    
    # Puisque p_i(mu) = 1/I pour tous i, on peut simplifier la somme
    sigma_p_sum = np.sum(sigma_values) / I  # C'est la somme de tous les sigma_i / I
    
    # Calcul de q_i*(mu) pour chaque i
    q_star = (sigma_values / sigma_p_sum) / I
    
    return q_star


In [96]:
qi = calculate_q_star(mu, 100, v, Sigma, 2000, results) #les allocations optimales qi 

Calcul des quantites Mi à allouer. 

In [95]:
def calculate_M(q, M):
    # Calculer la somme cumulative des qj
    cumulative_sums = [0] * len(q)
    cumulative_sums[0] = q[0]
    for i in range(1, len(q)):
        cumulative_sums[i] = cumulative_sums[i - 1] + q[i]

    # Calculer Mi pour chaque i
    Mi = []
    for i in range(len(q)):
        if i == 0:
            sum_j_less_i = 0  # Pas de j < i pour i = 0
        else:
            sum_j_less_i = cumulative_sums[i - 1]

        sum_j_less_equal_i = cumulative_sums[i]
        Mi.append(M * (sum_j_less_equal_i - sum_j_less_i))

    return Mi


In [98]:
calculate_M(qi,1000) #donne le nb de simu par strat. 

[14.664945500041554,
 11.2138435693111,
 11.028816970549943,
 10.947595939319593,
 10.671516338382329,
 10.666555840897567,
 10.630488084587581,
 10.525339145537924,
 10.408123918807105,
 10.540377384114313,
 10.287121996014225,
 10.35263758754526,
 10.35871381918732,
 10.238558856787483,
 10.203202530522038,
 10.150403858896178,
 10.19006371763881,
 10.199563193999866,
 10.16818795141225,
 9.96813880588887,
 9.785860889378245,
 9.912882434123693,
 10.141015663777647,
 10.123362626236421,
 10.042288598142212,
 9.837502700179911,
 9.89390406861923,
 9.953695905086713,
 9.953880155952943,
 9.811548855664153,
 9.847012396918597,
 9.863122779132095,
 9.746695199162103,
 9.76363236131994,
 9.880293787206618,
 9.80248432727282,
 9.952273639258824,
 9.818791950001572,
 9.678735632688173,
 10.110779658974723,
 9.869045910657404,
 9.770663399063073,
 9.681518167197535,
 9.793621765646188,
 9.690196752675128,
 9.885811895915309,
 9.67989610715475,
 9.585708661972003,
 9.62823609131991,
 9.654458

In [100]:
def initialisation(M, mu, I, v, Sigma, K=2000):
    """Args : 
        - M : nb de simu total pour le sample 
        - mu, Sigma : moyenne et variance de Y
        - v : direction de stratifaction 
        - K : nb de simu juste pour calculer les grandeurs intermediares tq sigma etc... """
    #renvoie ce qu'il faut pour étape 2 : à savoir les Mi, les pi, et la direction v de stratifaction 
    results = strats_Si(I) #les strats 
    qi = calculate_q_star(mu, I, v, Sigma, K, results)
    
    return calculate_M(qi,M), np.array(I*[1/I]), v

In [104]:
# Example usage:
mu = np.array([1, 2, 3])               # Mean vector - on est en 3 dimensions 
Sigma = np.array([[1, 0.5, 0.1],       # Covariance matrix
                  [0.5, 2, 0.3],
                  [0.1, 0.3, 1]])
v = np.array([1, 1, 1]) 

initialisation(10000, mu, 100, v, Sigma) #semble cohérent 

([146.79119134868,
  111.79922722794855,
  108.7903159941285,
  108.88689341342642,
  106.83567672601708,
  107.03012906611069,
  105.69765590631732,
  106.20392887006169,
  102.98742092957391,
  104.37825698219952,
  103.99844496252503,
  103.38920287717961,
  103.16452313917645,
  103.21256137615697,
  101.0214240135951,
  102.81728448861548,
  100.0633293429512,
  102.20256921311233,
  100.0581318148297,
  100.6439460973746,
  99.3512991469353,
  99.75743964485834,
  100.62794342382581,
  98.6414563844007,
  100.00950535246855,
  100.32007279146937,
  98.02722756044102,
  98.76546576923828,
  99.66611966462702,
  97.41097453906666,
  99.89687497245325,
  98.70484543257119,
  97.83407719602732,
  99.92225031413781,
  98.96587502356446,
  97.61270606099659,
  98.10886538186958,
  97.99435010590717,
  99.0033658053735,
  97.93519082687274,
  97.90650013910607,
  98.22696801811692,
  99.78474948291849,
  98.21448397849986,
  97.08555498695925,
  96.01044562659366,
  96.79795602099195,
 

In [112]:

def create_positive_semi_definite_matrix(dim):
    # Générer une matrice aléatoire
    A = np.random.randn(dim, dim)
    
    # Utiliser le produit de A et sa transposée pour obtenir une matrice semi-définie positive
    B = np.dot(A, A.T)
    
    # Normaliser les diagonales pour que la diagonale soit 1
    D = np.sqrt(np.diag(B))
    B = B / D
    B = B / D[:, np.newaxis]
    
    return B

# Créer une matrice de corrélation semi-définie positive de dimension 5x5
dim = 5
correlation_matrix = create_positive_semi_definite_matrix(dim)


[[ 1.          0.55205317  0.73311816 -0.7293417  -0.71632636]
 [ 0.55205317  1.          0.59653067 -0.24711168 -0.57553556]
 [ 0.73311816  0.59653067  1.         -0.72296239 -0.56856735]
 [-0.7293417  -0.24711168 -0.72296239  1.          0.18675538]
 [-0.71632636 -0.57553556 -0.56856735  0.18675538  1.        ]]


In [114]:
# Example usage:
mu = np.array([1, 2, 3,4,1])               # Mean vector - on est en 3 dimensions 
Sigma = correlation_matrix #on peut changer par ce qu'on veut ! 
v = np.array([1, 1, 1,1,1]) 

initialisation(10000, mu, 100, v, Sigma) #semble cohérent 

([169.893256699709,
  131.28186789427895,
  125.29806530501106,
  119.88408818741532,
  117.67504812426812,
  115.90647475985955,
  114.12136004781317,
  112.71556669299035,
  110.83430723488941,
  108.64002796651285,
  108.25844616046062,
  106.4419866265906,
  106.67398882004936,
  105.33391755338018,
  104.99077207465041,
  104.50998907915788,
  103.51407527326717,
  102.15275700983433,
  101.39995024297022,
  100.12049590625128,
  100.08336841371701,
  100.48209731419594,
  98.20464968624349,
  98.73752155498794,
  98.37010150861857,
  98.54504923305585,
  99.03591084717223,
  96.3861863405452,
  96.51873168977254,
  96.38845878783108,
  95.37563930869631,
  96.13940140146826,
  95.65459668564758,
  94.46783810413639,
  95.95625241132,
  93.86497445139086,
  93.88101527894766,
  94.6160990242162,
  94.32978282317761,
  92.57479342626773,
  93.64504574875254,
  93.0986509414844,
  94.0443900182486,
  92.87934038610723,
  93.33138423220811,
  92.04013480485551,
  92.44964359781393,
 